### move-семантика (since C++11)

<br />

##### Rationale

**Скорость**: решается проблема лишних копирований

```c++
// before C++11
std::vector<int> a = make_items(); // если компилятор не смог в NRVO, RVO, будет копирование
                                   // а можно бы и не копировать, объект всё равно временный.

// before C++11
std::vector<int> b;
a = b; // b больше не нужен, программист уверен,
       // зачем копировать контент b и сразу его уничтожать,
       // почему бы сразу контент не переиспользовать?
```

```c++
// since C++11
std::vector<int> a = make_items(); // no copy

// since C++11
std::vector<int> b;
a = std::move(b);  // no copy
```

**Удобство**: можно возвращать некопируемые объекты

```c++
// since C++11

// std::unique_ptr is non-copyable type
std::unique_ptr<Person> create_person();

// std::thread is non-copyable type
std::thread make_worker_thread();
```

<br />

##### Theory: lvalue, rvalue and more

https://devblogs.microsoft.com/cppblog/rvalue-references-c0x-features-in-vc10-part-2/

https://en.cppreference.com/w/cpp/language/value_category

http://www.stroustrup.com/terminology.pdf

http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2006/n2027.html

[Доклад "Understanding value categories in C++"](https://youtu.be/liAnuOfc66o)

Полное описание с длинным списком и исключениями можно посмотреть по ссылке на cppreference, здесь ниже будет дано словесное описание и опущены некоторые детали (типа функций, bitfields ...)

**C++98/03:**

> C++03 3.10/1: “Every expression is either an lvalue or an rvalue.”

<u>Важно для понимания: lvalue/rvalue - свойство выражений, а не объектов</u>


* `lvalue` - описывают объекты, которые остаются жить за рамками выражения. Примеры:

    ```c++
    obj
    *ptr
    ptr[index]
    ++x
    f() // T& f()
    ```
    
    Признак lvalue: если выражение представляет собой имя, то это однозначно lvalue.
    
    Обратное неверно: `ptr[index]` собой имя не представляет, но это тоже lvalue.
    

* `rvalue` - временные сущности, которые испаряются сразу в конце выражения (например, за точкой-с-запятой). Примеры:

    ```c++
    42
    x + y
    std::string("abc")
    x++
    f() // T f()
    ```

<br />

И `lvalue`, и `rvalue` выражения могут быть изменяемые (non-const) или неизменяемые (const). Примеры:

```c++
string       one(“cute”);
const string two(“fluffy”);
string       three() { return “kittens”; }
const string four() { return “are an essential part of a healthy diet”; }

one;     // mutable lvalue
two;     // const   lvalue
three(); // mutable rvalue
four();  // const   rvalue  <-- no practical sense
```

Правилами С++ установлено, что:
* Изменяемую ссылку (`Type&`) можно привязать только к mutable lvalue:

    ```c++
    string& ref = one;     // ok
    string& ref = two;     // compile error, only to mutable lvalue
    string& ref = three(); // compile error, only to mutable lvalue
    string& ref = four();  // compile error, only to mutable lvalue
    ```
    
* Неизменяемую ссылку (`const Type&`) можно привязать к чему угодно:

    ```c++
    const string& ref = one;     // ok
    const string& ref = two;     // ok
    const string& ref = three(); // ok
    const string& ref = four();  // ok
    ```

<u>Важно для понимания:</u> В записи:

```c++
{
    const std::string& ref = three();
    print_something(ref);
}
```

`three()` - это rvalue-выражение (т.к. временный объект), а `ref`- это lvalue-выражение (потому что дано имя). Когда возвращаемое значение функции как выражение привязывается к `const Type&`-ссылке, время жизни временного объекта продлевается до конца блока.

**Замечание**: обычно не рекомендуется писать код в стиле

 ```c++
const std::string& ref = three();
print_something(ref);
```
    
Потому что это сбивает с толку.

Теперь вы понимаете (теоретически), почему такой код по стандарту не компилируется:
    
```c++
void func(std::string& s);

func("hohoho");
```

А такой компилируется:

```c++
void func_1(std::string& s);

std::string s = "hohoho";
func_1(s);

//---

void func_2(const std::string& s);

func_2("hohoho");
```

<br />

**C++11/17:**

![](value_categories.png)

В С++11 у *выражения* вводится 2 свойства: has identity && can be moved from:

* `has identity` - у результата таких выражений можно взять адрес и сравнением адресов узнать, один и тот же это объект или нет.
* `can be moved from` - к выражению можно забиндить move-конструктор, move-assignment или другую функцию, реализующую move-семантику

Категории:

* `lvalue` = `has identity` + `can not be moved from`
    ```c++
    ++a;
    a += 3;
    *ptr;
    "hello world"
    f()  // T& f()
    ```
* `xvalue` (expiring value) = `has identity` + `can be moved from`
    ```c++
    std::move(s)  // string s;
    f()  // T&& f()
    ```
* `prvalue` = `has no identity` + `can be moved from`
    ```c++
    a + b
    f()  // T f()
    ```
* `glvalue` (generalized lvalue) = `has identity`
* `rvalue` = `can be moved from`

**Замечание:** это образное понимание категорий. Строгих и понятных правил, под какую категорию попадает выражение, нет. Вариант определения из [документации](https://en.cppreference.com/w/cpp/language/value_category): перечислить всевозможные выражения и <s>методом ленинского прищура</s> экспертным взглядом комитета прописать им категорию. Если обратите внимание на раздел "Defect reports" в документации, то видно, что не всегда экспертный взгляд комитета работал безошибочно.

<br />

##### Поддержка move-операций классами

move-операции должны оставить объект в валидном состоянии (возможно, unspecified)

**Вопрос:**

<details>
<summary>Что значит "валидное состояние"?</summary>
<p>
Соблюдены инварианты класса
</p>
</details>

Попробуем организовать move-операции для простенького класса строки:

```c++
class string {
public:
    string();
    ~string();
    
    string(const string& rhs);  
    string& operator =(const string& rhs);
    
    string(string&& rhs);
    string& operator =(string&& rhs);

private:
    char *s;
    size_t n;
};
```

Начнём с move constructor:

```c++
// move c-tor
string::string(string&& rhs)
    : s(rhs.s)
    , n(rhs.n)
{
    rhs.s = nullptr;
    rhs.n = 0;
}
```

**Вопрос:**

<details>
<summary>Корректен ли он?</summary>
<p>

`rhs` должен остаться в валидном состоянии. Класс должен поддерживать опцию `s == nullptr, n == 0` как реализацию пустой строки (во всех своих операциях)

</p>
</details>

Теперь move assignment operator

```c++
// move assignment operator
// option 1
string& string::operator = (string&& rhs) {
    std::swap(s, rhs.s);
    std::swap(n, rhs.n);
    return *this;
}

// move assignment operator
// option 2
string& string::operator = (string&& rhs) {
    s = rhs.s;
    n = rhs.n;
    rhs.s = nulltr;
    rhs.n = 0;
    return *this;
}
```

**Вопросы:**

* Какой вариант лучше? Чем?

<details>
<summary>ответ</summary>
<p>
    
В варианте 2 memory leak у `this->s`. А если его пофиксить:

1. вариант 1 работает для любой реализации строки, но потребляет лишнюю память
2. вариант 2 требует поддержать реализацию `s == nullptr, n == 0`, но потребление памяти меньше

</p>
</details>

* Какая неточность допущена в реализации второго варианта?

<details>
<summary>ответ</summary>
<p>

лучше поддержать безопасный вариант self-assigment: `x = std::move(x)`.

почему "лучше", но не обязательно?

</p>
</details>


Исправленный вариант 2:

```c++
// move assignment operator
// option 2, better
string& string::operator = (string&& rhs) {
    if (this != &rhs)
    {
        s = rhs.s;
        n = rhs.n;
        rhs.s = nulltr;
        rhs.n = 0;
    }
    return *this;
}
```

**Вопрос:**

* Чего ещё не хватает в реализации?

<details>
<summary>ответ</summary>
<p>
    
`noexcept` у move-операций. Напомните, зачем?

</p>
</details>

Исправленный вариант 2:

```c++
// move assignment operator
// option 2, better
string& string::operator = (string&& rhs) noexcept {
    if (this != &rhs)
    {
        s = rhs.s;
        n = rhs.n;
        rhs.s = nulltr;
        rhs.n = 0;
    }
    return *this;
}
```

<br />

##### автогенерированные move -операции

https://en.cppreference.com/w/cpp/language/move_constructor

https://en.cppreference.com/w/cpp/language/move_assignment

* Компилятор генерирует move constructor сам, если пользователь НЕ определил ни одну из операций: copy constructor, copy assignment operator, move assignment operator, destructors
* Компилятор генерирует move assignment сам, если пользователь НЕ определил ни одну из операций: copy-constructor, move constructor, copy assignment, destructor

**Вопрос:** Конечно же, все эти нюансы помнить не надо, потому что есть правило 6. А что такое правило 6?

С автогенерируемыми move-операциями следует быть крайне осторожными:
* они выполняют move по полям
* компилятор не заботится о наличии и сохранении инвариантов

Пример:

```c++
class ForwardListWithSizeCaching
{
private:
    std::forward_list<int> items;
    size_t count;
    
public:
    // auto generated move constructor
    ForwardListWithSizeCaching(ForwardListWithSizeCaching&& rhs)
        : items(std::move(rhs.items))  // likely rhs.items is empty after c-tor call
        , count(std::move(rhs.count))  // likely rhs.count is unchanged after c-tor call
    {
    }
    // Автогенерированный move-конструктор нарушает инварианты
    // внутри rhs, он остаётся сломан и пользоваться им нельзя
};
```

А здесь всё хорошо:

```c++
struct Person
{
    std::string name;
    std::string surname;
    std::uint32_t age;
};
```

<br />

##### Упражнение

Что будет вызвано?

```c++
class string {
public:
    string();
    string(const string& rhs);
    string(string&& rhs);
    string& operator =(const string& rhs);
    string& operator =(string&& rhs);
    ~string();
private:
    char *s;
    size_t n;
};

std::string         get_obj();
std::string&        get_ref();
std::string&&       get_exp_obj();
const std::string   get_cobj();
const std::string&  get_cref();
const std::string&& get_exp_cobj();

string s1;                 // ???
string s(s1);              // ???
string s{s1};              // ???
s = s1;                    // ???
string s = get_obj();      // ???
string s = get_ref();      // ???
string s = get_exp_obj();  // ???
string s = get_cobj();     // ???
string s = get_cref();     // ???
string s = get_exp_cobj(); // ???
```

<br />

##### move-операции и POD-типы

move-операции для POD-типов бесмысленны:

```c++
int get_number();

const int x = get_number(); // no so much win
```

```c++
struct Color
{
    char r;
    char g;
    char b;
};


// well ...
struct Color
{
    ...
        
    // not so much win
    Color(Color&& rhs)
        : r(std::move(rhs.r))
        , g(std::move(rhs.g))
        , b(std::move(rhs.b))
    {}
};
```

Случаи, в которых move-операции имеют смысл:

* когда можно реализовать более эффективный перенос ресурсов:

    ```c++
    std::string
    std::vector
    ```
    
* когда можно переиспользовать более эффективный move:

    ```c++
    struct Person
    {
        std::string name;
        std::string surname;
        std::uint32_t age;
    };
    ```
    
* когда копирование запрещено:

    ```c++
    std::unique_ptr
    std::thread
    ```

<br />

##### std::move

<u>std::move does not move!</u>

Как реализован `std::move`:

```c++
template<class T>
inline constexpr typename remove_reference<T>::type&& move(T&& t) noexcept
{
    return static_cast<typename remove_reference<T>::type&&>(t);
}
```

Чтобы было проще прочитать, введём сокращение:

`TV = typename remove_reference<T>::type` - удаление `&` и `&&` у типа `T`.

И спрячем модификаторы `inline`, `constexpr`, `noexcept`, т.к. они принципиально на понимание не влияют:

```c++
template<class T>
TV&& move(T&& t)
{
    return static_cast<TV&&>(t);
}
```

Т.о. `std::move` принимает ссылку на объект и пытается вернуть rvalue-ссылку:

* `string&` -> `string&&`
* `string&&` -> `string&&`
* `const string&` -> `const string&&`
* `const string&&` -> `const string&&`

С самим объектом ничего не происходит.

**Вопрос на понимание зачем нужен std::move:** Пусть есть строка `string s`. Какого типа выражения `s` и `std::move(s)`?

<details>
<summary>ответ</summary>
<p>

`s` - lvalue (т.к. выражение - имя)

`std::move(s)` - xvalue (см. примеры)

Обратите внимание, что объект один и тот же, но поменялся тип выражения, который его обозначает.

</p>
</details>

<br />

Примеры использования `std::move`:

```c++
std::string s1 = ...;
std::string s2 = s1;             // s1 unused below
std::string s2 = std::move(s1);  // s1 unused below
// вопрос: какое состояние s1 здесь?

struct Person
{
    std::string name;
    std::string surname;
    
    ...
    Person(Person&& rhs);
};

Person::Person(Person&& rhs)
    : name(rhs.name)        // rhs.name is lvalue, copy c-tor called
    , surname(rhs.surname)  // rhs.surname is lvalue, copy c-tor called
{
}

Person::Person(Person&& rhs)
    : name(std::move(rhs.name))        // std::move(rhs.name) is xvalue, move c-tor called
    , surname(std::move(rhs.surname))  // std::move(rhs.surname) is xvalue, move c-tor called
{
}
```

<br />

**Отступление про ссылки и категории**

Рассмотрим функцию:

```c++
void func(std::string&& name) { /*...*/ }
```

Функция может принимать только rvalue - выражения. Варианты её вызова:

```c++
// prvalue
std::string make_name();
func(make_name());

// xvalue
std::string city_name = "Novosibirsk";
func(std::move(city_name));
```

В функции:
* `name` - это rvalue-reference, т.е. ссылка на временный объект
* `name` - как выражение - lvalue (почему?)

поэтому внутри функции, если мы хотим срабатывание move-операций, нужно делать так:

```c++
void func(std::string&& name)
{
    std::string full_name = name;
    std::string full_name = std::move(name);  // std::move здесь конвертирует lvalue-выражение в xvalue-выражение,
                                              // оба выражения обозначают один и тот же объект - rvalue-reference to temporary
    /*...*/
}
```

<br />

##### Упражнения с std::move

Что будет вызвано? Как преобразует тип выражения `std::move`?

```c++
class string {
public:
    string();
    string(const string& rhs);
    string(string&& rhs);
    string& operator =(const string& rhs);
    string& operator =(string&& rhs);
    ~string();
private:
    char *s;
    size_t n;
};

std::string         get_obj();
std::string&        get_ref();
std::string&&       get_exp_obj();
const std::string   get_cobj();
const std::string&  get_cref();
const std::string&& get_exp_cobj();

string s1;                            // ???
const string& cs1 = s1;               // ???
string s(std::move(s1));              // ???
string s = std::move(s1);             // ???
s = std::move(s1);                    // ???
string s = std::move(cs1);            // ???
string s = std::move(get_obj());      // ???
string s = std::move(get_ref());      // ???
string s = std::move(get_exp_obj());  // ???
string s = std::move(get_cobj());     // ???
string s = std::move(get_cref());     // ???
string s = std::move(get_exp_cobj()); // ???
```

<br />

##### стандартные ошибки

**Ошибка 1**: self-move-assignment для типов, реализацию которых вы не контроллируете

```c++
std::vector<int> v = ...;
v = std::move(v);
```

v - unspecified valid state (скорее всего, пустой)

**Ошибка 2**: std::move(pod)

```c++
struct Color
{
    unsigned char r;
    unsigned char g;
    unsigned char b;
};

Color c = ...;
Color x = std::move(c);
```

или так:

```c++
std::string s1 = “run, Forest, run!”;
std::string s2;
std::move(s1.begin(), s1.end(), std::back_inserter(s2));  // алгоритм, выполняющий std::move поэлементно
```

**Ошибка 3**: std::move(const)

```c++
void say_my_name(const std::string& name)
{
    std::string res = std::move(name);  // copy
    ...
}
```

<br />

##### передача параметров в функцию, метод, конструктор

Задача: есть функция `f`, её нужно принять строку `s` как параметр. Как принимать? (про perfect forwarding сейчас не говорим)

* Если `s` не меняется и не копируется, то очень просто:

```c++
void f(const std::string& s);
```

* Если `s` меняется или копируется - всё становится несколько интереснее

Рассмотрим варианты:

```c++
void f(const std::string& s) { auto sc = s; ... }  // by const lvalue reference
void f(std::string s);         // by value
void f(std::string&& s);       // by mutable rvalue reference
```

И варианты вызова:

```c++
std::string name;
std::string make_name();

f(name);             // lvalue
f(std::move(name));  // xvalue
f(make_name());      // prvalue
```



**Упражнение:** Заполним таблицу, какие операции вызываются:


| ~                    | `void f(const string& s) { auto sc = s; ... }` | `void f(string s)` | `void f(string&& s)`  |
|:---------------------|:----------------------------------------------:|:------------------:|:---------------------:|
| `f(name)`            |                                                |                    |                       |
| `f(std::move(name))` |                                                |                    |                       |
| `f(make_name())`     |                                                |                    |                       |


<br />

**Замечание:** как организовать вызов `f(string&& s)` от lvalue-выражения:

```c++
string s = "abc";
f(string(s));
```

<br />

**Ответ:**

| ~                    | `void f(const string& s) { auto sc = s; ... }` | `void f(string s)` | `void f(string&& s)`  |
|:---------------------|:----------------------------------------------:|:------------------:|:---------------------:|
| `f(name)`            | copy                                           | copy               | x (* copy+ugly stx)   |
| `f(std::move(name))` | copy                                           | move               |                       |
| `f(make_name())`     | copy                                           | move               |                       |


В случае, если строку нужно копировать или менять внутри функции:

```c++
void f(string&& s);  // рекомендовано cppcoreguidelines
void f(string s);    // обычно рекомендуют в coding styles компаний
```

<br />

Теперь мы хотим передать параметр в конструктор, чтобы инициализировать поле класса:
    
```c++
class Person
{
private:
    std::string name;

public:
    Person(const std::string& n) : name(n) {}             // by const lvalue reference
    Person(std::string n)        : name(std::move(n)) {}  // by value
    Person(std::string&& n)      : name(std::move(n)) {}  // by mutable rvalue reference
};
```

И проделаем аналогичное упражнение:
    
| ~                      | `Person(const string& n)` | `Person(string n)`    | `Person(string&& n)`  |
|:-----------------------|:-------------------------:|:---------------------:|:---------------------:|
| `Person(s)`            |                           |                       |                       |
| `Person(std::move(s))` |                           |                       |                       |
| `Person(make_name())`  |                           |                       |                       |

**Ответ:**

| ~                      | `Person(const string& n)` | `Person(string n)`    | `Person(string&& n)`  |
|:-----------------------|:-------------------------:|:---------------------:|:---------------------:|
| `Person(s)`            | copy                      | copy + move           | x                     |
| `Person(std::move(s))` | copy                      | move + move           | move                  |
| `Person(make_name())`  | copy                      | move + move           | move                  |


Для инициализации полей класса:

```c++
Person(string&& s);  // рекомендовано cppcoreguidelines
Person(string s);    // обычно рекомендуют в coding styles компаний
```

<br />

##### Perfect forwarding && universal references

https://isocpp.org/blog/2012/11/universal-references-in-c11-scott-meyers

Для peformance-критичных участков кода хочется:
* избежать даже лишних move-ов при передаче аргументов в функцию / конструктор, поэтому вариант `Person(string n)` не подходит.
* но при этом писать адекватный код для lvalue-выражений

Добавим в класс `Person` фамилию и помотрим как это можно сделать:

```c++
class Person
{
private:
    string name_;
    string surname_;
    
public:
    Person(const string&  name, const string&  surname) : name_(name),            surname_(surname)            {}
    Person(const string&  name,       string&& surname) : name_(name),            surname_(std::move(surname)) {}
    Person(      string&& name, const string&  surname) : name_(std::move(name)), surname_(surname)            {}
    Person(      string&& name,       string&& surname) : name_(std::move(name)), surname_(std::move(surname)) {}
};
```

А потом добавим в `Person` отчество и вечер начинает быть томным.

<br />

Вариант решения проблемы - perfect forwarding && universal references - особая запись особого типа. Сначала рассмотим на примере обычной функции от одного аргумента, потом вернёмся к `Person`:

```c++
template<typename T>
void f(T&& t) { ... }
```

<u>Важный момент:</u> `T&&` конкретно в такой записи - это НЕ rvalue-reference, это universal reference!

*Комитет делает всё возможное чтобы посильнее запутать программистов (нет), но вы должны быть грамотными!*

Как это работает:

* когда компилятор встречает вызов `f` от lvalue-выражения, он генерирует вариант `f(T&)` или `f(const T&)`
* когда компилятор встречает вызов `f` от rvalue-выражения, он генерирует вариант `f(T&&)`

Закинуть этот код на godbolt.org и продемонстрировать сколько функций сгенерировал компилятор. Не забыть `-O0 (gcc trunk)`:

```c++
#include <utility>

template<typename T>
void some_function(T&& t) {}

int main()
{
    int x = 0;
    some_function(x);

    some_function(int(x));

    const int& cref = x;
    some_function(cref);
}
```

<br />

Возможная реализация `f`:

```c++
template<typename T>
T turn_fury_on(T&& t)
{
    // make object to modify (either copy either move)
    T rv(std::forward<T>(t));
    
    // modify object
    for (auto& c : rv)
        c = to_upper(c);
    
    return rv;
}

turn_fury_on(std::string("performance matters!"));  // PERFORMANCE MATTERS!, move inside + NRVO

std::string s = "performance matters!";
turn_fury_on(s);  // PERFORMANCE MATTERS!, copy inside + NRVO
```

`std::forward` - специальный утилитарный однострочник в стандартной библиотеке, который помогает идеально передать параметры внутри функции. Выглядит он, например, так:

```c++
template<class T>
T&& forward(typename std::remove_reference_t<T>& t) noexcept {
  return static_cast<T&&>(t);
}

template <class T>
T&& forward(typename std::remove_reference_t<T>&& t) noexcept {
  return static_cast<T&&>(t);
}
```

Или для конкретного типа после разворачивания шаблонов и подстановки universal reference:

```c++
string& forward(string& t) noexcept {
  return static_cast<string&>(t);
}

string&& forward(string&& t) noexcept {
  return static_cast<string&&>(t);
}
```

<br />

Тогда реализация perfect forwarding для класса `Person`:

```c++
class Person
{
private:
    string name_;
    string surname_;
    
public:
    template<typename NameT, typename SurnameT>
    Person(NameT&& name, SurnameT&& surname)
        : name_(std::forward<NameT>(name))
        , surname_(std::forward<SurnameT>(surname))
    {}
    // обратите внимание, что под name и surname
    // выделены отдельные типы в шаблоне, т.к.
    // один может быть lvalue, другой - rvalue
};
```

и добавить отчество в класс уже не так страшно

<br />

**Замечание:** тип аргумента должен быть именно `Type&&`:

```c++
template<typename T>
void f(T&& t);               // OK - universal reference

template<typename T>
void f(std::vector<T>&& t);  // !!! rvalue reference

template<typename T>
void f(const T&& t);         // !!! rvalue reference
```

<br />

##### Резюме

* move-семантика нужная для:
    * скорости, чтобы избавиться от лишних копирований
    * корректности, чтобы можно было возвращать некопируемые объекты
* категории выражений, которые желательно понимать и различать обычным смертным: `lvalue`, `xvalue`, `prvalue`
* после выполнения move-операции над классом он остаётся в unspecified valid state (за некоторыми отдельно прописанными исключениями типа `std::unique_ptr`, `std::shared_ptr`, кастомных классов и т.п.)
* автогенерированные move-операции работают по полям и могут нарушать инварианты, будьте внимательны
* POD-типы move-ать бесмысленно
* Ooops:

    ```c++
    vector<int> v(...);
    v = std::move(v);
    ```

* `std::move`:
    * std move does not move, это изменение категории выражения
    * std move чаще всего нужен, чтобы -mutable lvalue "скастовать в rvalue":
    
    ```c++
    std::string s = ...;
    v.push_back(std::move(s));
    ```
    
* если нужно изменять передаваемый в функцию объект, то использовать передачу:

    ```c++
    f(string s);
    f(string&& s)
    ```

* perfect forwarding - способ оптимально передавать параметры